In [1]:
!pip install transformers
!pip install scikit-learn
!pip install torch

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import math
print("[INF] base loaded")
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
print("[INF] torch loaded")
from transformers import BertTokenizer, BertModel
print("[INF] transformers loaded")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[INF] base loaded
[INF] torch loaded


[INF] transformers loaded


In [3]:
print("[INF] parsing dataset...")

csv_path = "ssh_anomaly_dataset.csv"
df = pd.read_csv(csv_path)

df["target"] = (df["label"] != "normal").astype(int)

df["timestamp"] = pd.to_datetime(df["timestamp"])
df["hour"] = df["timestamp"].dt.hour
df["minute"] = df["timestamp"].dt.minute

df = df.drop(columns=["label", "timestamp", "detail"])

X_data = df.drop(columns=["target"])
y_data = df["target"]

tmpX = (X_data["hour"].astype(str)
    + " " + X_data["minute"].astype(str)
    + " " + X_data["source_ip"]
    + " " + X_data["username"]
    + " " + X_data["event_type"]
    + " " + X_data["status"])
X_data = tmpX.tolist()

categorical_cols = ["source_ip", "username", "event_type", "status"]
numeric_cols = ["hour", "minute"]

[INF] parsing dataset...


In [4]:
print("[INF] creating tokenizer...")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bertModel = BertModel.from_pretrained("bert-base-uncased")
bertModel.eval()
bertModel.to(device)

[INF] creating tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
def bert_embeddings(texts):
    batch_size = 512
    embeddings = []
    attention_mask = None
    with torch.no_grad():
        inputs = tokenizer(texts, return_tensors='pt', truncation=True, max_length=70, padding='max_length')
        attention_mask = inputs["attention_mask"].cpu()

        print("[INF] tokens ready, embedding...")
        number_of_batches = len(inputs["input_ids"]) // batch_size
        for i in tqdm(range(number_of_batches)):
            batch_inputs = { k: v[batch_size * i : batch_size * (i + 1)] for k, v in inputs.items() }
            batch_inputs_on_device = { k: v.to(device) for k, v in batch_inputs.items() }
            outputs = model(**batch_inputs_on_device)

            # embeddings.append(outputs.last_hidden_state.squeeze().cpu())
            embeddings.append(outputs.last_hidden_state[:, 0, :].squeeze().cpu())

        last_batch = len(inputs["input_ids"]) // batch_size * batch_size
        print("[INF] last batch...")
        batch_inputs = { k: v[last_batch : ] for k, v in inputs.items() }
        batch_inputs_on_device = { k: v.to(device) for k, v in batch_inputs.items() }
        outputs = model(**batch_inputs_on_device)

        # embeddings.append(outputs.last_hidden_state.squeeze().cpu())
        embeddings.append(outputs.last_hidden_state[:, 0, :].squeeze().cpu())

        # outputs = model(**inputs)
    # for text in tqdm(texts):
    #     with torch.no_grad():
    #         inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=70, padding='max_length')
    #         outputs = model(**inputs)

    #         embeddings.append(outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy())
    # embeddings = np.array(embeddings, axis=1)

    # embeddings = np.concatenate(embeddings)
    embeddings = torch.cat(embeddings)
    return embeddings, attention_mask

print("[INF] Creating embeddings...")
embeddings, attention_mask = bert_embeddings(X_data)

print(embeddings[0].shape)
print(y.shape)

X_train, X_test, mask_train, mask_test, y_train, y_test = train_test_split(
    embeddings,
    attention_mask,
    y_data,
    test_size=0.2,
    random_state=42,
    stratify=y_data
)


In [ ]:
del embeddings, attention_mask

In [ ]:
class EmbeddingTransformer(nn.Module):
    def __init__(self, embedding_dim=768, num_classes=2):
        super().__init__()

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=8,
            dim_feedforward=1024,
            batch_first=True
        )

        self.encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=2
        )

        self.classifier = nn.Linear(embedding_dim, num_classes)

    def forward(self, x, attention_mask):
        pad_mask = attention_mask == 0

        x = self.encoder(x, src_key_padding_mask=pad_mask)

        attention_mask = attention_mask.unsqueeze(-1)
        pooled = (x * attention_mask).sum(dim=1) / attention_mask.sum(dim=1)

        return self.classifier(pooled)

model = EmbeddingTransformer().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(10):
    optimizer.zero_grad()

    train_ds = TensorDataset(
        X_train.to(device),
        mask_train.to(device),
        torch.tensor(y_train.values).to(device))
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

    total_loss = 0
    for x, mask, y in tqdm(train_loader):
        print(x, mask.shape)
        logits = model(x, mask)
        loss = loss_fn(logits, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss / len(train_loader):.4f}")

    # logits = model(X_train.to(device), mask_train.to(device))
    # loss = loss_fn(logits, y_train)

    # loss.backward()
    # optimizer.step()

    # print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")

  0%|          | 0/523 [00:00<?, ?it/s]

tensor([[-0.4471,  0.0879,  0.0793,  ..., -0.3476,  0.0111,  0.4947],
        [-0.3937,  0.0811,  0.0748,  ..., -0.4169,  0.0334,  0.7187],
        [-0.4851,  0.1455,  0.2456,  ..., -0.4474,  0.3873,  0.8037],
        ...,
        [-0.3139,  0.0932,  0.0735,  ..., -0.3640,  0.2653,  0.6025],
        [-0.3456,  0.1297,  0.0069,  ..., -0.3339,  0.0676,  0.6689],
        [-0.4577,  0.1225,  0.0556,  ..., -0.4428,  0.0943,  0.6686]],
       device='cuda:0') torch.Size([64, 70])


AssertionError: For unbatched (2-D) `query`, expected `key_padding_mask` to be `None` or 1-D but found 2-D tensor instead

In [ ]:
model.eval()
with torch.no_grad():
    preds = model(X_test.to(device), mask_test.to(device)).argmax(dim=1)
    acc = (preds.cpu() == y_test).float().mean()

print("Test accuracy:", acc.item())

In [5]:
def bert_embeddings(texts):
    with torch.no_grad():
        inputs = tokenizer(texts, return_tensors='pt', truncation=True, max_length=70, padding='max_length')
        tokenized_texts = inputs["input_ids"].cpu()
        attention_mask = inputs["attention_mask"].cpu()
        return tokenized_texts, attention_mask

print("[INF] Creating tokenized inputs...")
tokenized_texts, attention_mask = bert_embeddings(X_data)

X_train, X_test, mask_train, mask_test, y_train, y_test = train_test_split(
    tokenized_texts,
    attention_mask,
    y_data,
    test_size=0.2,
    random_state=42,
    stratify=y_data
)


[INF] Creating tokenized inputs...


In [38]:
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=128):
        super().__init__()

        # self.embedding = nn.Embedding(
        #     vocab_size,
        #     d_model,
        #     padding_idx=tokenizer.pad_token_id
        # )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=4,
            dim_feedforward=256,
            batch_first=True
        )

        self.encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=2
        )

        self.classifier = nn.Linear(d_model, len(pd.unique(y_data)))

    def forward(self, input_ids, attention_mask):
        pad_mask = (attention_mask == 0).to(device)

        output_ids = bertModel(input_ids, attention_mask)
        x = output_ids.last_hidden_state.squeeze().cpu().to(device)
        # x = output_ids.last_hidden_state[:, 0, :].squeeze().cpu()

        # x = self.embedding(input_ids)
        x = self.encoder(x, src_key_padding_mask=pad_mask)

        attention_mask_unsqueeze = attention_mask.unsqueeze(-1)
        x = (x * attention_mask_unsqueeze).sum(dim=1) / attention_mask_unsqueeze.sum(dim=1)

        pred = self.classifier(x)
        del x, pad_mask
        return pred

model = SimpleTransformer(vocab_size=tokenizer.vocab_size, d_model=768)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [37]:
for epoch in range(10):
    model.train()
    optimizer.zero_grad()

    # train_ds = TensorDataset(
    #     X_train.to(device),
    #     mask_train.to(device),
    #     torch.tensor(y_train.values).to(device))
    train_ds = TensorDataset(
        X_train,
        mask_train,
        torch.tensor(y_train.values))
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

    its = 1
    total_loss = 0
    for x, mask, y in tqdm(train_loader, desc=f"{total_loss/its:0.4f}"):
        x = x.to(device)
        mask = mask.to(device)
        y = y.to(device)

        logits = model(x, mask)
        loss = loss_fn(logits, y)

        del x, mask, y

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        its += 1

    print(f"Epoch {epoch+1} | Loss: {total_loss / len(train_loader):.4f}")


0.0000:   3%|▎         | 14/523 [00:11<06:48,  1.25it/s]


KeyboardInterrupt: 

In [32]:
torch.cuda.empty_cache()

In [39]:
model.eval()
test_ds = TensorDataset(
    X_test,
    mask_test,
    torch.tensor(y_test.values))
test_loader = DataLoader(test_ds, batch_size=64, shuffle=True)
total_acc = 0.0
with torch.no_grad():
    for x, mask, y in tqdm(test_loader):
        X_test_device = x.to(device)
        mask_test_device = mask.to(device)
        preds = model(X_test_device, mask_test_device).argmax(dim=1)
        acc = np.sum((preds.cpu() == y).numpy()) / len(y)
        # acc = (preds.cpu() == y_test).float().mean()
        del X_test_device, mask_test_device

        print(acc)
        total_acc += acc

print()
print("Test accuracy:", total_acc / len(test_loader))


  1%|          | 1/131 [00:00<00:39,  3.29it/s]

0.9375


  2%|▏         | 2/131 [00:00<00:36,  3.49it/s]

0.890625


  2%|▏         | 3/131 [00:00<00:35,  3.65it/s]

...

0.953125


 98%|█████████▊| 129/131 [00:36<00:00,  3.81it/s]

0.890625


100%|██████████| 131/131 [00:36<00:00,  3.55it/s]

0.859375
0.8222222222222222

Test accuracy: 0.911453032230704
